In [1]:
BIZPATH='../montreal_subset/montreal_business.json'
USERPATH='../montreal_subset/montreal_users_who_reviewed.json'
REVPATH='../montreal_subset/montreal_reviews.json'
PARTIAL_DIM_PATH = '../montreal_subset/featureset1_but_votes_Restaurants.json'

In [2]:
import json

In [3]:
bizrecords = [json.loads(line) for line in open(BIZPATH)]
userrecords = [json.loads(line) for line in open(USERPATH)]
revrecords = [json.loads(line) for line in open(REVPATH)]
pdimrecords = [json.loads(line) for line in open(PARTIAL_DIM_PATH)]

In [4]:
from pandas import DataFrame, Series

In [5]:
import pandas as pd; import numpy as np

In [6]:
bizframe = DataFrame(bizrecords)
userframe = DataFrame(userrecords)
revframe = DataFrame(revrecords)
pdimframe = pd.read_json(PARTIAL_DIM_PATH)

In [7]:
bizframe_sub = DataFrame(bizrecords, columns=['business_id', 'name', 'categories', 'review_count', 'stars'])

In [8]:
rev_user_frame = pd.merge(revframe, userframe, on="user_id")
rev_biz_frame = pd.merge(revframe, bizframe, on="business_id")

In [9]:
#Months yelping
import datetime
now = datetime.datetime.now()
userframe['months_yelping'] = userframe['yelping_since'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m')).map(lambda x: (now.year - x.year) * 12 + now.month - x.month)

In [10]:
userframe.head()

,average_stars,compliments,elite,fans,friends,name,review_count,type,user_id,votes,yelping_since,months_yelping
0,3.52,"{u'profile': 5, u'funny': 37, u'cute': 8, u'pl...","[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",102,"[rpOyqD_893cqmDAtJLbdog, CYJnkecyF1DOXVFvBeFmt...",Matthew,1820,user,e-t3dRzH9bmsanmhYgSiVQ,"{u'funny': 2020, u'useful': 6438, u'cool': 2835}",2006-05,118
1,3.85,"{u'profile': 8, u'funny': 28, u'cute': 8, u'pl...","[2013, 2014, 2015]",117,"[rpOyqD_893cqmDAtJLbdog, 8J4IIYcqBlFch8T90N923...",James,1444,user,uF1-c9oji5qiHntYsmfKGg,"{u'funny': 4040, u'useful': 7417, u'cool': 5710}",2011-11,52
2,3.47,"{u'cute': 1, u'cool': 1, u'funny': 1, u'photos...",[],1,[snd45oedwXxF9tsHUX_6dg],Meg,44,user,NUNVFFv_VEyVn3G7kg1V8g,"{u'funny': 21, u'useful': 83, u'cool': 14}",2009-09,78
3,3.45,"{u'profile': 2, u'cute': 4, u'funny': 37, u'pl...","[2009, 2010, 2011, 2012]",22,"[uguXfIEpI65jSCH5MgUDgA, Rir-YRPPClKXDFQbc3BsV...",Jeff,394,user,0KZsDw5pGKrYEWGXg9oUsg,"{u'funny': 956, u'useful': 1524, u'cool': 970}",2008-05,94
4,3.39,"{u'profile': 1, u'funny': 4, u'cute': 1, u'pla...","[2006, 2007, 2009, 2010, 2011]",15,"[GJYJX4SujVj3BR8v2F9PDQ, WGf64PeXIZtdoqkBj0-_z...",Susan,482,user,gMOlCdsMxODvZ9G7yMZmTA,"{u'funny': 164, u'useful': 530, u'cool': 206}",2006-02,121


In [11]:
#Label elite users
userframe["elite_user"] = userframe['elite'].map(lambda x: 'elite' if len(x) > 0 else 'not_elite')


In [12]:
#Adapted from: http://stackoverflow.com/questions/21160134/flatten-a-column-with-value-of-type-list-while-duplicating-the-other-columns-va
#TODO: Find more idiomatic definition for rev_category_frame, preferably by removing flattened_categories.
flattened_categories = DataFrame([[i, category]
                                     for i, categories in rev_biz_frame['categories'].apply(list).iteritems()
                                         for category in categories],
                                columns=['index', 'category'])
flattened_categories.set_index('index')
rev_category_frame = rev_biz_frame.merge(flattened_categories, left_index = True, right_index = True)

In [13]:
#Number businesses / category
#TODO: Add name to the size column; do some sanity checking on the data.
user_biz_revved_per_category_frame = rev_category_frame.groupby(['user_id', 'business_id', 'category']).size()

In [14]:
rev_restaurants_frame = DataFrame(rev_category_frame.loc[rev_category_frame['category'] == 'Restaurants'])
rev_restaurants_frame['funny'] = rev_restaurants_frame['votes'].map(lambda x: x['funny'])
rev_restaurants_frame['useful'] = rev_restaurants_frame['votes'].map(lambda x: x['useful'])
rev_restaurants_frame['cool'] = rev_restaurants_frame['votes'].map(lambda x: x['cool'])

In [15]:
rev_dim_peruser_frame = rev_restaurants_frame.groupby('user_id').aggregate(sum).reset_index()[['user_id', 'funny', 'useful', 'cool']]

In [16]:
user_dim_frame = pd.merge(userframe[['user_id', 'months_yelping', 'elite_user']], rev_dim_peruser_frame, on='user_id')
dimframe = pd.merge(pdimframe, user_dim_frame, on='user_id')

In [17]:
#Move elite_user column to end
elite_column = dimframe.pop('elite_user')
dimframe.insert(10,'elite_user', elite_column)

In [18]:
dimframe.to_json('../montreal_subset/featureset1.json')

In [19]:
dimframe.head()

,categories_avg_rating,categories_biz_count,categories_reviews,std_dev_rating,total_reviews,user_id,months_yelping,funny,useful,cool,elite_user
0,4,2,2,0,6,Vi6yC7XiZDADYG73Z0MJAQ,28,0,1,0,elite
1,4,2,2,0,6,Vi6yC7XiZDADYG73Z0MJAQ,28,0,1,0,elite
2,4,2,2,0,6,Vi6yC7XiZDADYG73Z0MJAQ,28,0,1,0,elite
3,4,2,2,0,6,Vi6yC7XiZDADYG73Z0MJAQ,28,0,1,0,elite
4,4,2,2,0,6,Vi6yC7XiZDADYG73Z0MJAQ,28,0,1,0,elite
